<center><h1>ResNet</h1></center>

<center><p><a href="https://arxiv.org/abs/1512.03385">Deep Residual Learning for Image Recognition</a></p></center>

<img src="https://production-media.paperswithcode.com/methods/Screen_Shot_2020-09-25_at_10.26.40_AM_SAB79fQ.png" width="600"/>

In [1]:
import torch
from torch import nn

# Blocks

## Conv Blocks

In [2]:
def conv1x1(in_channels, out_channels, stride=1) -> nn.Conv2d:
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)


def conv3x3(in_channels, out_channels, stride=1) -> nn.Conv2d:
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

## Residual Networks (ResNet-18, ResNet-34)

<img src="https://www.jeremyjordan.me/content/images/2018/04/Screen-Shot-2018-04-16-at-5.47.38-PM.png" width="600"/>

In [3]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv3x3(in_channels, out_channels, stride=stride)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = conv3x3(out_channels, out_channels, stride=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample if downsample is not None else nn.Identity()

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        identity = self.downsample(identity)
        out += identity
        out = self.relu(out)

        return out

## Deeper Bottleneck Architectures (ResNet-50, ResNet-101, ResNet-152)

<img src="https://www.jeremyjordan.me/content/images/2018/04/Screen-Shot-2018-04-16-at-5.47.38-PM.png" width="600"/>

In [4]:
class Bottleneck(nn.Module):
    # self.conv1: conv1x1(stride=stride) -> conv1x1(stride=1)
    # self.conv2: conv3x3(stride=1) -> conv3x3(stride=stride)
    # This improves the model by 0.5%.
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv1x1(in_channels, out_channels, stride=1)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = conv3x3(out_channels, out_channels, stride=stride)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.conv3 = conv1x1(out_channels, out_channels * self.expansion, stride=1)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample if downsample is not None else nn.Identity()

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        identity = self.downsample(identity)
        out += identity
        out = self.relu(out)

        return out

# ResNet

Half the image size in conv1(7x7), max pool(3x3), conv3/4/5_2(3x3, differ from the paper which use 1x1 conv3/4/5_1).

Expand the channel size in conv2/3/4/5_3(3x3) if needed.

In [5]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super().__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # 64->64->64 -> (64->64->64)*N
        # 64->64->64->256 -> (256->64->64->256)*N
        self.layer1 = self._make_layer(block, 64, layers[0])
        # 64->128->128 -> (128->128->128)*N
        # 256->128->128->512 -> (512->128->128->512)*N
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        # 128->256->256 -> (256->256->256)*N
        # 512->256->256->1024 -> (1024->256->256->1024)*N
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        # 256->512->512 -> (512->512->512)*N
        # 1024->512->512->2048 -> (512->512->512->2048)*N
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        self._initialize_weights(zero_init_residual)

    def forward(self, x):
        x = self.conv1(x)  # 64, H/2, W/2
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)  # 64, H/4, W/4

        x = self.layer1(x)  # 64/256, H/4, W/4
        x = self.layer2(x)  # 128/512, H/8, W/8
        x = self.layer3(x)  # 256/1024, H/16, W/16
        x = self.layer4(x)  # 512/2048, H/32, W/32

        x = self.avgpool(x)  # 512/2048, 1, 1
        x = torch.flatten(x, 1)  # 512/2048
        x = self.fc(x)  # 1000

        return x

    def _make_layer(self, block, channels, block_num, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != channels * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.in_channels, channels * block.expansion, stride=stride),
                nn.BatchNorm2d(channels * block.expansion),
            )

        layers = [block(self.in_channels, channels, stride=stride, downsample=downsample)]
        self.in_channels = channels * block.expansion
        for _ in range(1, block_num):
            layers.append(block(self.in_channels, channels))

        return nn.Sequential(*layers)

    def _initialize_weights(self, zero_init_residual=False):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)

# Summary

## Data

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

data = torch.randn((32, 3, 224, 224)).to(device)

## ResNet-18

In [7]:
from torchkeras import summary

net = ResNet(
    block=BasicBlock,
    layers=[2, 2, 2, 2],
    num_classes=1000,
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                          [-1, 64, 112, 112]                9,408
BatchNorm2d-2                     [-1, 64, 112, 112]                  128
ReLU-3                            [-1, 64, 112, 112]                    0
MaxPool2d-4                         [-1, 64, 56, 56]                    0
Conv2d-5                            [-1, 64, 56, 56]               36,864
BatchNorm2d-6                       [-1, 64, 56, 56]                  128
ReLU-7                              [-1, 64, 56, 56]                    0
Conv2d-8                            [-1, 64, 56, 56]               36,864
BatchNorm2d-9                       [-1, 64, 56, 56]                  128
Identity-10                         [-1, 64, 56, 56]                    0
ReLU-11                             [-1, 64, 56, 56]                    0
Conv2d-12                           [

## ResNet-34

In [8]:
net = ResNet(
    block=BasicBlock,
    layers=[3, 4, 6, 3],
    num_classes=1000,
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                          [-1, 64, 112, 112]                9,408
BatchNorm2d-2                     [-1, 64, 112, 112]                  128
ReLU-3                            [-1, 64, 112, 112]                    0
MaxPool2d-4                         [-1, 64, 56, 56]                    0
Conv2d-5                            [-1, 64, 56, 56]               36,864
BatchNorm2d-6                       [-1, 64, 56, 56]                  128
ReLU-7                              [-1, 64, 56, 56]                    0
Conv2d-8                            [-1, 64, 56, 56]               36,864
BatchNorm2d-9                       [-1, 64, 56, 56]                  128
Identity-10                         [-1, 64, 56, 56]                    0
ReLU-11                             [-1, 64, 56, 56]                    0
Conv2d-12                           [

## ResNet-50

In [10]:
net = ResNet(
    block=Bottleneck,
    layers=[3, 4, 6, 3],
    num_classes=1000,
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                          [-1, 64, 112, 112]                9,408
BatchNorm2d-2                     [-1, 64, 112, 112]                  128
ReLU-3                            [-1, 64, 112, 112]                    0
MaxPool2d-4                         [-1, 64, 56, 56]                    0
Conv2d-5                            [-1, 64, 56, 56]                4,096
BatchNorm2d-6                       [-1, 64, 56, 56]                  128
ReLU-7                              [-1, 64, 56, 56]                    0
Conv2d-8                            [-1, 64, 56, 56]               36,864
BatchNorm2d-9                       [-1, 64, 56, 56]                  128
ReLU-10                             [-1, 64, 56, 56]                    0
Conv2d-11                          [-1, 256, 56, 56]               16,384
BatchNorm2d-12                     [-

## ResNet-101

In [12]:
net = ResNet(
    block=Bottleneck,
    layers=[3, 4, 23, 3],
    num_classes=1000,
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                          [-1, 64, 112, 112]                9,408
BatchNorm2d-2                     [-1, 64, 112, 112]                  128
ReLU-3                            [-1, 64, 112, 112]                    0
MaxPool2d-4                         [-1, 64, 56, 56]                    0
Conv2d-5                            [-1, 64, 56, 56]                4,096
BatchNorm2d-6                       [-1, 64, 56, 56]                  128
ReLU-7                              [-1, 64, 56, 56]                    0
Conv2d-8                            [-1, 64, 56, 56]               36,864
BatchNorm2d-9                       [-1, 64, 56, 56]                  128
ReLU-10                             [-1, 64, 56, 56]                    0
Conv2d-11                          [-1, 256, 56, 56]               16,384
BatchNorm2d-12                     [-

## ResNet-152

In [14]:
net = ResNet(
    block=Bottleneck,
    layers=[3, 8, 36, 3],
    num_classes=1000,
).to(device)

summary(net, input_data=data)
del net

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                          [-1, 64, 112, 112]                9,408
BatchNorm2d-2                     [-1, 64, 112, 112]                  128
ReLU-3                            [-1, 64, 112, 112]                    0
MaxPool2d-4                         [-1, 64, 56, 56]                    0
Conv2d-5                            [-1, 64, 56, 56]                4,096
BatchNorm2d-6                       [-1, 64, 56, 56]                  128
ReLU-7                              [-1, 64, 56, 56]                    0
Conv2d-8                            [-1, 64, 56, 56]               36,864
BatchNorm2d-9                       [-1, 64, 56, 56]                  128
ReLU-10                             [-1, 64, 56, 56]                    0
Conv2d-11                          [-1, 256, 56, 56]               16,384
BatchNorm2d-12                     [-